In [7]:
!pip install langchain
!pip install langchain-community
!pip install langchain-chroma
!pip install langchain-text-splitters
!pip install python-git
# !pip install -U langchain langchain-community

In [ ]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts import ChatPromptTemplate
import os
from git import Repo

In [9]:
#https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval.create_retrieval_chain.html
# from langchain_community.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub

In [ ]:
os.environ['OPENAI_API_KEY'] = ""

In [10]:
repo_path = "./dir_repo"

In [11]:
repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)

In [14]:
loader = GenericLoader.from_filesystem(
    repo_path + r"\\libs\\core\\langchain_core",
    glob="**/*",
    suffixes=[".py"],
    exclude=["**/non-utf-8-encoding.py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),
)

In [15]:
documents = loader.load()

In [16]:
len(documents)

470

### Separando em chunks

In [19]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=1000, chunk_overlap=200
)

In [20]:
texts = python_splitter.split_documents(documents)

In [21]:
len(texts)

2683

### Fazendo o embedding e salvando no banco de vetores

In [24]:
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))

In [25]:
retriver = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8},
)

- LLM

In [29]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [30]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "Você é um revisor de código experitente. Forneça informações detalhadas sobre a revisão do código e sugestões de melhorias baseado no contexto fornecido abaixo: \n\n {context} "
        ),
        (  "user",  "{input}")
    ]
)

In [31]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [33]:
retrieval_chain = create_retrieval_chain(
    retriver,
    document_chain,
)

In [36]:
response = retrieval_chain.invoke({"input": "Você pode revisar e sugerir melhorias para o código de RunnableBinding"})

In [37]:
print(response)

{'input': 'Você pode revisar e sugerir melhorias para o código de RunnableBinding', 'context': [Document(metadata={'content_type': 'functions_classes', 'language': 'python', 'source': 'dir_repo\\libs\\core\\langchain_core\\runnables\\base.py'}, page_content='class RunnableBinding(RunnableBindingBase[Input, Output]):\n    """Wrap a Runnable with additional functionality.\n\n    A RunnableBinding can be thought of as a "runnable decorator" that\n    preserves the essential features of Runnable; i.e., batching, streaming,\n    and async support, while adding additional functionality.\n\n    Any class that inherits from Runnable can be bound to a `RunnableBinding`.\n    Runnables expose a standard set of methods for creating `RunnableBindings`\n    or sub-classes of `RunnableBindings` (e.g., `RunnableRetry`,\n    `RunnableWithFallbacks`) that add additional functionality.\n\n    These methods include:'), Document(metadata={'language': 'python', 'source': 'dir_repo\\libs\\core\\langchain_co

In [40]:
print(response["answer"])

### Revisão do Código:

1. A classe `RunnableBinding` parece estar bem estruturada e documentada, o que é um bom sinal.
2. A documentação fornece uma boa visão geral do propósito da classe e como ela se relaciona com outras classes no sistema.
3. A classe parece estar focada em adicionar funcionalidades adicionais a um `Runnable` existente, mantendo suas características essenciais.

### Sugestões de Melhoria:

1. **Implementação de Métodos Abstratos:** A classe `RunnableBinding` parece ser uma classe base genérica. Seria útil adicionar métodos abstratos ou pelo menos métodos de exemplo para mostrar como a classe deve ser estendida e implementada.
   
2. **Exemplos de Uso:** Adicionar exemplos de uso da classe `RunnableBinding` pode ajudar os desenvolvedores a entender melhor como ela deve ser utilizada na prática.

3. **Remoção de Códigos Comentados:** Há trechos de código comentados que não parecem ser necessários. Recomenda-se remover esses trechos para manter o código limpo e mais f